This notebook calculates the sediment contributing drainage area in the year 2025 for all of ResNet using the methods of Minear and Kondolf (xxxx).

In [ ]:
#import packages
import pandas as pd
import numpy as np
import os

In [ ]:
#load data

# ResNet File location and name
today = datetime.today().strftime('%Y%m%d')
resnet = pd.read_csv(f'Outputs/ResNet_{today}.csv')

#load canadian dams for calculating
canada = pd.read_csv('Inputs/InputCanada.csv')

#output file location
out_folder = 'Outputs' 

In [ ]:
#conversions
convert1 = 1233.482 #converts m3 to ac-ft is convert1*AF=m3, or from m3 is m3/convert1=AF
convert2 = 2.59 #converts km2 and mi2 is convert2*mi2=km2 or from km2 is km2/convert2=mi2

In [ ]:
#converts canada Batch_for to cell. Is this necessary?

#combine Canada and resnet
resnet = pd.concat([resnet,canada])

resnet = resnet.sort_values(by='ShortID',ascending = True)

# All matlab below here for now

In [ ]:
#changing river nan to 0; maybe not necessary? and i think shouldn't exist?
%remove nans from rivers
a=isnan(data.isriver);
b=find(a==1);
data.isriver(b)=0;
clear a; clear b;

In [ ]:
%% Fix routing columbia river dams to route major canada dams
% Grand Coulee:
% remove 78916 & 117361 from "From Dam"
% add to from dam:  500002, 500004, 500006
% set D50 to whatever grand coulee is

boundary=find(data.SID==117361); %Boundary Dam
data.todam(boundary)=500005; data.grandtag(boundary)=500005; %Should go to canadian Dam, not directly to Grand Coulee

libby=find(data.SID==78196); % Libby Dam
data.todam(libby)=500003; data.grandtag(libby)=500003;  %Should go to canadian Dam, not directly to Grand Coulee

coulee=find(data.SID==117548);
tmp = cellfun(@eval, data.fromdam(coulee), 'UniformOutput', false);
couleefrom=cell2mat(tmp);
junk=find(couleefrom==data.SID(libby)| couleefrom==data.SID(boundary));
couleefrom(junk)=[];
a=numel(couleefrom)+1;
b=a+2;
couleefrom(a:b)=([500002 500004 500006]);
test=num2str([couleefrom]);
test2=['[' test ']'];
data.fromdam{coulee}=test2; %does work. 

%just use same D50 as downstream
canada=find(data.SID>=500000);
data.D50(canada)=data.D50(coulee);

vars={'boundary','libby','coulee','couleefrom','a','b','junk','tmp','canada', 'test','test2','InputAll','InputCAN','InputNID','inputdate'};
clear(vars{:}); clear vars;

In [ ]:
%% % transfer delta IDs to delta tags (weren't tagged if it was the downstream river mouth). Aaron- Probably can delete this from the code. but if there is no where to go "to" deltas weren't tagged as deltas
a=find(data.deltaID~=0 & data.SID<0);
data.DeltaTag(a)=data.deltaID(a);
clear a;


In [ ]:
%% CLEAN SITE INPUT STARTS HERE

#converting cells; shouldn't be necessary
data.flag   = cellfun(@eval, data.flag, 'UniformOutput', false);
data.fromdam = cellfun(@eval, data.fromdam, 'UniformOutput', false);


#maybe unnecessary and can still treat as nan
%change year removed from 0 to the future
a=isnan(data.yrr); 
junk=find(a==1);
data.yrr(junk)=3001;
remove0=find(data.yrr==0);
data.yrr(remove0)=3001;

#changes nan todam to 0. again, can probably ignore.
% check todam
notodam=isnan(data.todam); %find where "ToDam category = NaN"
junk=find(notodam==1); %change the NaN to 0
data.todam(junk)=0;
clear junk; clear notodam;

In [ ]:
%% Remove rivers from sites, todam, sitetags etc. Aaron some of these data checks can be deleted if a new resnet run doesn't flag them

%remove rivers/deltas from the "To Dam" category. May need to loop through
%only once if downstream dams have their "ToDam" replaced before the
%upstream dam in encountered in the i=1:imax loop

numdamstoriver=find(data.todam<0);
countnum=numel(numdamstoriver);
while countnum>0
    for i=1:countnum
        %find a dam that goes to a river location instead of another dam
        damlocationtofix=numdamstoriver(i); % this is the location in the full dataset
        theriver=data.todam(damlocationtofix); %this is the SID of the river it goes to
        riverlocation=find(data.SID==theriver); % this is the location of the river it goes to in the full dataset 
        todamofriver=(data.todam(riverlocation)); %this is "to dam" SID of the downstream river
        %replace the "to dam" of the target dam with the "to dam" of the river"
        data.todam(damlocationtofix)=todamofriver;
    end
    numdamstoriver=find(data.todam<0);
    countnum=numel(numdamstoriver);
end

In [ ]:
%Make it so that rivermouths aren't considered sites
rivers=find(data.isriver==1);
data.issite(rivers)=0;

In [ ]:
% identify "headwater" rivers (rivers that were flagged like headwater dams). If they go to a dam, then the dam then needs
% to be potentially reclassified as a headwater dam, if not downstream from another dam on antoher flowline. 
headrivers=find(data.isriver==1 & data.head==1);
fromHR=data.todam(headrivers);
realdams=find(fromHR>0);

% if isempty(realdams)~=1
%     realdamlocation=find(data.SID==fromHR(realdams));
%     ErrorStop="Check these dams, see if they need to be reclassified as headwater dams"
%     return
% end

% now that to dam has been replaced on real dams, remove to dam from rivers
data.todam(rivers)=NaN; 

% fix term dam flagging after removing rivers
notterm=find(data.todam>0); %would have been 
data.term(notterm)=0;
term=find(data.todam==0);
data.term(term)=1; %makes these terminal dams
data.term(rivers)=NaN;


In [ ]:
%% Create additional river tags for rivers downstream from rivers - Aaron you can probably skip this unless you wanted to add to your code

%first change is river NaN to 0
a=isnan(data.isriver);
itsnan=find(a==1);
data.isriver(itsnan)=0;
clear a; clear itsnan;

NIDnum=length(data.SID);
tags(:,1)=data.rivertag1;
j=1;
tag=find(tags(:,j)~=0);
str="rivertag";

%Make a rivertags cell
data.rivertags=cell(NIDnum,1);
for i=1:NIDnum
data.rivertags(i)={[data.rivertag1(i)]}; 
end

while isempty(tag)~=1
    tags(:,(j+1))=zeros(NIDnum,1);
    for i=1:numel(tag)
        taglocation=tag(i);%row within array "tags column j" that has a site tag
        SIDoftaggedriver=tags(taglocation,j);
        taggedriverlocation=find(data.SID==SIDoftaggedriver); %row of the tagged site
        tags(taglocation,(j+1))=tags(taggedriverlocation,1);%found original location of the river- its tag will be in column 1, if one exists.
    end
    j=j+1;
    varname=append(str,num2str(j));
    data.(varname)=tags(:,j);
    temp=data.(varname);
    tag=find(tags(:,j)~=0);   
    for i=1:numel(tag)
        idx=tag(i);
        rts_mat=cell2mat(data.rivertags(idx));
        new=numel(rts_mat)+1;
        rts_mat(new)=temp(idx);
        data.rivertags(idx)={rts_mat}; 
    end
    clear temp; clear varname; clear rts_mat; clear new; clear idx;

end


In [ ]:
%%  Make a ranking file of dam order Aaron delete the data checks ErrorStop
%Make a ranking file 

Rank=NaN(NIDnum,1);
head=find(data.head==1);
Rank(head)=1; clear head;
Rank(rivers)=0; clear rivers;

DAerrorNumber=0;
i=1;
ranknum=find(Rank==i);

while isempty(ranknum)~=1
    jmax=numel(ranknum); % number of dams with a rank of i
    for j=1:jmax
        thisdam=ranknum(j); % The index we are on within array j
        if  data.term(thisdam)==0 % if dam j is not a terminal dam, then
            thisdamDA=data.DA(thisdam); % identify DA of j
            goesto=data.todam(thisdam); % identify where dam j goes to
            thatdam=find(data.SID==goesto); % find the SID of the dam that dam j goes to
            thatdamDA=data.DA(thatdam); % identify the DA that dam j goes to

            %do a check to make sure downstream DA doesn't exceed upstream
            if thatdamDA<thisdamDA %identify if upstream drainage area is greater than downstream drainage area
                DAerrorNumber=DAerrorNumber+1;
                n=DAerrorNumber;
            else
                Rank(thatdam)=Rank(thisdam)+1;
            end
        end
    end
    i=i+1;
    ranknum=find(Rank==i);
end

% if DAerrorNumber~=0
%     ErrorStop="Manually investigate DA errors in GIS and find solution"
%     return
% end
data.Rank=Rank;
vars={'Rank','check','DAerrorNumber','goesto','i','j','n','ranknum','thatdam','thatdamDA','thisdam','thisdamDA','jmax','NIDnum'};
clear(vars{:}); clear vars;


In [ ]:
%% Get Rid of Rivers 
junk=find(data.isriver==1);
rivers=data(junk,:);
data(junk,:)=[];

data= removevars(data,{'isriver'});


In [ ]:

%% Fix yrc, replace with 90th percentile of when dams were built- could probably delete the 2025 line above and combine this.

data.yrc_SUB=0*data.yrc;
nocomp=find(data.yrc<1700 | data.yrc>2024);
numdam=numel(data.yrc);
realdate=find(data.yrc>0 & data.yrc<2025);
yr_sub=prctile(data.yrc(realdate),90); % if no completion date, using the 90th percentile of yrc to sub in a date (1991). 50th percent is approx 1963. 75th percent is approx 1974, 90th percentile is approx 1991, 99th percentile is approx 2011;
data.yrc(nocomp)=yr_sub;
data.yrsub=zeros(numdam,1);
data.yrsub(nocomp)=1; %identifys dams where we assumed a yrc date


In [ ]:
%% Fill in year prediction & capp- abby loop uses capp, so leave for now- this uses CAPP.  not sure what aaron needs.  may need to just set capp = capc or change code to have capc
%sometimes a survey shows a growth in capacity- dam raise, or better survey technique etc. We have these data for some
%Usace and usbr sites. here, the prediction year (yrp) is the newer survey.  Alternatively, sometimes there was no original survey
% and the first survey was collected later. We will back calculate between yrp and yrc. 
%if no yrp, then yrp=yrc
% cap p is capacity at year p

% fill in yrp where it doesn't exist (non sites with no specified yrp)
junk1=isnan(data.yrp);
junk2=find(data.yrp==0);
junk3=find(junk1==1);
junk=vertcat(junk2,junk3);
data.yrp(junk)=data.yrc(junk); %set the empty yrp as same as yrc. 
clear junk; clear junk1; clear junk2; clear junk3;

%replace maxstor with prediction capacity at locations were we have a capp.
%these will be reclamation and usace sites
a=find(data.capp>0 & data.issite==0);
data.maxstor(a)=data.capp(a);
clear a;

%maxstor is capc at sites or capp at non sites with later surveys.
% now capp can be set to equal maxstor.
data.capp=data.maxstor;

%change perm storage from NaN to 1
junk=isnan(data.PermStorag);
junk1=find(junk==1);
data.PermStorag(junk1)=1;
clear junk; clear junk1;


In [ ]:
%% MLR function input starts here

% Create variables and empty datasets

%create the timeseries
t=[1699:1:2050];
numdam=numel(data.SID);
numt=numel(t);

%redoing calcs with no changing trap efficiencies
capcalc=NaN(numdam,numt); %this is a variable that would need to be stored in the structure 
sedshed=NaN(numdam,numt); %this is the km2 area of the watershed that has sediment getting trapped in reservoir (so contribut DA * trap efficiency)
calctrap=zeros(numdam,numt);
wsedshed=NaN(numdam,1);% Time weighted (between surveys) SAatDam above REservoir X * calctrap at dam X for Reservoir X ("effective sediment contributing DA")
origDA=NaN(numdam,numt); %this is the original Drainage area through time 
    jmax=numdam;
    for j=1:jmax
        origDA(j,:)=data.DA(j);
    end
wSAatdam=NaN(numdam,1); %Time weighted (between surveys) sediment-contributing drainage area above Reservoir X 
AveTrap=NaN(numdam,1); %time weighted trap efficiency (between surveys)- used to constrain sediment delivery
wseddel=NaN(numdam,1); %m3, total volume of sediment delivered to reservoir X between survey 1 and survey 2
wSDR=NaN(numdam,1);%m3/yr, Sediment delivery rate, mean volume of sediment delivered to reservoir X per year between surveys
wSDRyield=NaN(numdam,1); %m3/(km3*t). sediment yeild. volume sed per km2 per yr         

% Calculate SA at Dam using initial TE
sedDAtoDS=NaN(numdam,numt); % the is the DA that moves downstream, start with NaN. will be overwritten
SAatdam=(origDA);% this is the sediment contributing drainage area upstream from a reservoir X (does not include trap efficiency at reservoir X).


In [ ]:
%% Trap efficiency

%calc kappa, coarse(sand) = 1, medium(silt)= 0.1, fine(clay)= 0.046;
%using design silt assumption
%kappa(:,1)=0.1; %design assumption; there are no null values
kappa=zeros(numdam,1)+0.1;

% trap efficiency everywhere is static to create a SAatDAM series for abby based on initial TE 
jmax=numdam; 
for j=1:jmax
    %assign trap efficiency (0) before dam completion
    predam=find(t<data.yrc(j));
    postdam=find((t>=(data.yrc(j)) & (t<data.yrr(j))));
    removed=find(t>=data.yrr(j));
    calctrap(j,predam)=0; capcalc(j,predam)=0;%predam, capacity and trap efficiency = 0 
    calctrap(j,removed)=0;capcalc(j,removed)=0; %post dam removal 
    origDA(j,:)=data.DA(j);
    calctrap(j,postdam)=1-1./(1+kappa(j)*(((data.maxstor(j))/convert1)./((data.DA(j))/convert2))); %stationary non-site trap efficiency
    capcalc(j,postdam)=data.maxstor(j);
    AveTrap(j,1)=calctrap(j,(postdam(1))); %since static just fills in the first trap efficiency
    clear postdam; clear predam;clear removed;
end
clear jmax; clear j;

imin=1;
imax=max(data.Rank);
for i=imin:imax
    ranknum=find(data.Rank==i);
    jmax=numel(ranknum);
    sedshed(ranknum,:)=SAatdam(ranknum,:).*calctrap(ranknum,:); %km2, this will calc sedshed except beyond second survey for the rank we are on
    sedDAtoDS(ranknum,:)=SAatdam(ranknum,:)-sedshed(ranknum,:); %km2, this is the volume moving downstream past dam site in any given year, up to second survey.
    for j=1:jmax
        val=ranknum(j);
        if data.term(val)==0 %if it isn't a term dam, move that drainage area downstream
            goesto=data.todam(val);
            thatdam=find(data.SID==goesto);
            SAatdam(thatdam,:)=SAatdam(thatdam,:)-(data.DA(val)-sedDAtoDS(val,:)); %Adjusts SA at dam for next time loop. Rank of this dam must be higher than dam it comes from
            %double check for any drainage area errors, can be caused by
            %flow diversions
            uhoh=find(SAatdam(thatdam,:)<0);
            clear goesto; clear thatdam; clear uhoh;
        end
        clear val;
    end
    clear ranknum; clear j; clear n;
end


In [ ]:
%% Save MLR TimesSeries Input...... you would want to change this file name for your export
%timeseries output, for MLR input
MLR_SAatdam_timeseries=NaN((numdam+1),(numt+1));
MLR_SAatdam_timeseries(1,2:end)=t;
MLR_SAatdam_timeseries(2:end,1)=data.SID;
MLR_SAatdam_timeseries(2:end,2:end)=SAatdam_TEyrc(:,:);
writematrix(MLR_SAatdam_timeseries,MLRfilename2,'Delimiter',',');

vars={'export','calctrap1','calctrap2','origDA','wsedshed','wSAatdam','wSDR','wseddel','tmax','numt','numdam','numsite' ...
    't','sedshed','calctrap','capcalc','SAatdam','sedDAtoDS','AveTrap','notsite','site','wSAatdam','wSDRyield','wseddel','wsedshed','kappa','calctrapp','convert1','convert2','data','exportfilename','ExportGISdata','MLRfilename','MLRfilename2','PredictGrand','Predictusace','Predictusbr','SAatdam_TEyrc','TotalPrediction','trap1','trap2'};
clear(vars{:}); clear vars;

clear i; clear imax; clear imin; clear j; clear jmax; clear jmin; clear n; clear nmin; clear nmax; clear val; 
clear T;
save MLRdata.mat
DoneMLRinput=1
